In [13]:
path = ".."
# get data from problem count from classification file

import os
import sys

sys.path.append(path)
total_qs = 0
total_type_1 = 0
total_type_2 = 0
total_type_3 = 0
for i in range(2003, 2024):
    try: 
        with open(f"{path}/iol_{i}/iol_{i}_classification.txt", "r") as f:
            data = f.readlines()
            count_data = data[-4:]
            total = int(count_data[0].split(":")[1].strip())
            type_1 = int(count_data[1].split(":")[1].strip())
            type_2 = int(count_data[2].split(":")[1].strip())
            type_3 = int(count_data[3].split(":")[1].strip())
            total_qs += total
            total_type_1 += type_1
            total_type_2 += type_2
            total_type_3 += type_3
            points = type_1 + type_2*2 + type_3*2
            print(f"Total points for {i}: {points}")
        # with open(f"{path}/iol_{i}/iol_{i}_classification.txt", "a") as f:
        #     f.write(f"\nTotal points: {points}")
    
    except:
        print(f"Error in {i}")

print(f"Total questions: {total_qs}")
print(f"Total type 1: {total_type_1}")
print(f"Total type 2: {total_type_2}")
print(f"Total type 3: {total_type_3}")

Error in 2003
Total points for 2004: 47
Total points for 2005: 60
Total points for 2006: 81
Total points for 2007: 35
Total points for 2008: 69
Total points for 2009: 65
Total points for 2010: 72
Total points for 2011: 57
Total points for 2012: 105
Total points for 2013: 44
Total points for 2014: 107
Total points for 2015: 126
Total points for 2016: 89
Total points for 2017: 61
Total points for 2018: 87
Total points for 2019: 142
Error in 2020
Total points for 2021: 99
Total points for 2022: 86
Total points for 2023: 82
Total questions: 1031
Total type 1: 546
Total type 2: 453
Total type 3: 31


In [2]:
import json
import re

def parse_grading_text_from_file(file_path):
    # Create a structure to hold the final result
    result = {}
    
    current_problem = None
    current_part = None
    current_question = None
    current_data = {}

    # Read the content from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Process each line
    for line in lines:
        line = line.strip()  # Clean leading/trailing spaces

        # Match "Problem X:"
        problem_match = re.match(r'^Problem (\d+):', line)
        if problem_match:
            current_problem = f"Problem {problem_match.group(1)}"
            result[current_problem] = {}
            current_part = None  # Reset current part and question when a new problem starts
            current_question = None
            continue

        # Match "Part X:" where X can be a letter or a number
        part_match = re.match(r'^Part ([a-zA-Z0-9]+):', line)
        if part_match:
            current_part = f"Part {part_match.group(1)}"
            result[current_problem][current_part] = {}
            current_question = None  # Reset question for this part
            continue

        # Ensure that there is a current problem and part before proceeding
        if not current_problem or not current_part:
            continue

        # Match question number (e.g., 1, 2, 3, ...) or detect the absence of a question number
        question_match = re.match(r'^(\d+):', line)
        if question_match:
            current_question = question_match.group(1)
            current_data = {}
            result[current_problem][current_part][current_question] = current_data
            continue

        # Handle case when no question number exists (direct Classification under Part)
        if not current_question:
            current_data = result[current_problem][current_part]
        
        # Match Classification, LLM Answer, Real Answer, and Grade
        classification_match = re.match(r'^Classification:\s*(.+)', line)
        llm_answer_match = re.match(r'^LLM Answer:\s*(.+)', line)
        real_answer_match = re.match(r'^Real Answer:\s*(.+)', line)
        grade_match = re.match(r'^Grade:\s*(\d+)', line)

        if classification_match:
            current_data["Classification"] = classification_match.group(1)
        elif llm_answer_match:
            current_data["LLM Answer"] = llm_answer_match.group(1)
        elif real_answer_match:
            current_data["Real Answer"] = real_answer_match.group(1)
        elif grade_match:
            current_data["Grade"] = int(grade_match.group(1))

    return result


In [3]:
import yaml
import json
import pandas as pd
from difflib import SequenceMatcher
from nltk.translate.bleu_score import sentence_bleu


# load to yaml helper func
def text_to_json(path):
    try:
        parsed_data = parse_grading_text_from_file(path)
        json_text = json.dumps(parsed_data, indent=4)
        return json.loads(json_text)
    except Exception as e:
        print(e)
        print(f"Error parsing text for path {path}")
        return None

# function to get scores and data from graded data
def grad_parser(graded_data: json):
    json_obj = graded_data
    total_score = 0
    total_possible = 0
    total_questions = 0
    type1_right = 0
    type1_possible = 0
    type2_right_b = 0
    type2_right_d = 0
    type2_right_a = 0
    type2_possible = 0
    type3_right = 0
    type3_possible = 0
    for problem in json_obj:
        if problem[0:7] != "Problem":
            continue
        total_questions += 1
        for part in json_obj[problem]:
            part_score = 0
            part_possible = 0
            for question in json_obj[problem][part]:
                if question[0:4] == "Clas":
                    try:
                        get_grade = int(json_obj[problem][part]['Grade'][0])
                    except:
                        get_grade = json_obj[problem][part]['Grade']
                        
                    if json_obj[problem][part]['Classification'] == "Type 1":
                        part_score += get_grade
                        type1_right += get_grade
                        part_possible += 1
                        type1_possible += 1
                    elif json_obj[problem][part]['Classification'] == "Type 2":
                        overlap_d = 2 * SequenceMatcher(None, json_obj[problem][part]['LLM Answer'], json_obj[problem][part]['Real Answer']).ratio()
                        # use bleu score instead
                        overlap_b = 2 * sentence_bleu([json_obj[problem][part]['Real Answer'].split()], json_obj[problem][part]['LLM Answer'].split())
                        part_score += overlap_d
                        part_possible += 2
                        type2_right_b += overlap_b
                        type2_right_a += get_grade
                        type2_right_d += overlap_d
                        type2_possible += 2
                    elif json_obj[problem][part]['Classification'] == "Type 3":
                        part_score += get_grade
                        part_possible += 2
                        type3_right += get_grade
                        type3_possible += 2
                    break
                # grading
                try:
                    get_grade = int(json_obj[problem][part][question]['Grade'][0])
                except:
                    get_grade = json_obj[problem][part][question]['Grade']
                    
                if json_obj[problem][part][question]['Classification'] == "Type 1":
                    part_score += get_grade
                    type1_right += get_grade
                    part_possible += 1
                    type1_possible += 1
                elif json_obj[problem][part][question]['Classification'] == "Type 2":
                    overlap_d = 2 * SequenceMatcher(None, json_obj[problem][part][question]['LLM Answer'], json_obj[problem][part][question]['Real Answer']).ratio()
                    overlap_b = 2 * sentence_bleu([json_obj[problem][part][question]['Real Answer'].split()], json_obj[problem][part][question]['LLM Answer'].split())
                    part_score += overlap_d
                    part_possible += 2
                    type2_right_b += overlap_b
                    type2_right_a += get_grade
                    type2_right_d += overlap_d
                    type2_possible += 2
                elif json_obj[problem][part][question]['Classification'] == "Type 3":
                    part_score += get_grade
                    part_possible += 2
                    type3_right += get_grade
                    type3_possible += 2
                    
            total_score += part_score
            total_possible += part_possible
    accuracy = total_score / total_possible
    print("(" + str(type3_possible) + ", " + str(type3_right) + "),")
    return total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right_a, type2_right_d, type2_right_b, type2_possible, type3_right, type3_possible

In [4]:
def vision_prob_parser(graded_data: json):
    json_obj = graded_data
    total_score = 0
    total_possible = 0
    total_questions = 0
    type1_right = 0
    type1_possible = 0
    type2_right = 0
    type2_possible = 0
    type3_right = 0
    type3_possible = 0
    for problem in json_obj:
        if problem[0:7] != "Problem":
            continue
        total_questions += 1
        for part in json_obj[problem]:
            part_score = 0
            part_possible = 0
            for question in json_obj[problem][part]:
                if question[0:4] == "Clas":
                    try:
                        get_grade = int(json_obj[problem][part]['Grade'][0])
                    except:
                        get_grade = json_obj[problem][part]['Grade']
                        
                    if json_obj[problem][part]['Classification'] == "Type 1":
                        part_score += get_grade
                        type1_right += get_grade
                        part_possible += 1
                        type1_possible += 1
                    elif json_obj[problem][part]['Classification'] == "Type 2":
                        part_score += get_grade
                        part_possible += 2
                        type2_right += get_grade
                        type2_possible += 2
                    elif json_obj[problem][part]['Classification'] == "Type 3":
                        part_score += get_grade
                        part_possible += 2
                        type3_right += get_grade
                        type3_possible += 2
                    break
                # grading
                try:
                    get_grade = int(json_obj[problem][part][question]['Grade'][0])
                except:
                    get_grade = json_obj[problem][part][question]['Grade']
                    
                if json_obj[problem][part][question]['Classification'] == "Type 1":
                    part_score += get_grade
                    type1_right += get_grade
                    part_possible += 1
                    type1_possible += 1
                elif json_obj[problem][part][question]['Classification'] == "Type 2":
                    part_score += get_grade
                    part_possible += 2
                    type2_right += get_grade
                    type2_possible += 2
                elif json_obj[problem][part][question]['Classification'] == "Type 3":

                    part_score += get_grade
                    part_possible += 2
                    type3_right += get_grade
                    type3_possible += 2
            total_score += part_score
            total_possible += part_possible
    accuracy = total_score / total_possible
    return total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right, type2_possible, type3_right, type3_possible

In [5]:
years = [str(i) for i in range(2004, 2024)]
# remove 2020
years.append('samples')
years.remove('2020')

models = ["gpt4o", "claude3.5s", "claude3o", "claude3h", "gemini1.5p", "gpt4", "gpt4om", "o1strawberry"]

eval_table = pd.DataFrame(columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible","Type 2 Right LLM", "Type 2 Right Diff", "Type 2 Right Bleu", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])

# add blank rows to eval_table
for year in years:
    for model in models:
        path = '../iol_' + year + '/' + model + '/evaluation.txt'
        if year == 'samples':
            path = '../iol_samples/text/' + model + '/evaluation.txt'
        json_obj = text_to_json(path)
        if json_obj:
            #print(f"Proc essing year {year} for model {model}")
            total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right_a, type2_right_d, type2_right_b, type2_possible, type3_right, type3_possible = grad_parser(json_obj)

            # if accuracy < 0.1:
            #     print(f"Processing year {year} for model {model}")
            #     print(f"Total Score: {total_score}/{total_possible}")
            #     print(f"Accuracy: {accuracy}")
            
            
            eval_table = pd.concat([eval_table, pd.DataFrame([[year, model, total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right_a, type2_right_d, type2_right_b, type2_possible, type3_right, type3_possible]], columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible", "Type 2 Right LLM", "Type 2 Right Diff", "Type 2 Right Bleu", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])], ignore_index=True)
        else:
            print(f"Error processing year {year} for model {model}")


/Users/satyamgoyal/anaconda3/envs/lm_mentor/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/satyamgoyal/anaconda3/envs/lm_mentor/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/satyamgoyal/anaconda3/envs/lm_mentor/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

(8, 1),
(8, 1),
(8, 3),
(8, 4),
(8, 3),
(8, 2),
(8, 4),
(8, 2),
(6, 4),
(6, 2),
(6, 4),
(6, 1),
(6, 4),
(6, 1),
(6, 1),
(6, 6),
(20, 10),
(20, 5),
(20, 8),
(20, 3),
(20, 10),
(20, 6),
(20, 11),
(20, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 2),
(2, 1),
(2, 2),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(4, 2),
(4, 2),
(4, 1),
(4, 0),
(4, 2),
(4, 1),
(4, 2),
(4, 2),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 0),
(2, 0),
(2, 1),
(6, 1),
(6, 2),
(6, 2),
(6, 4),
(6, 1),
(6, 2),
(6, 0),
(6, 1),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(4, 2),
(4, 3),
(4, 3),
(4, 2),
(4, 3),
(4, 2),
(4, 2),
(4, 1),
(4, 2),
(4, 1),
(4, 4),
(4, 1),
(4, 2),
(4, 1),
(4, 3),
(4, 1),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 1),
(2, 0),
(2, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0

In [6]:
type3right_manual = [(8, 1),
(8, 1),
(8, 3),
(8, 4),
(8, 3),
(8, 2),
(8, 4),
(8, 4),
(6, 4),
(6, 2),
(6, 4),
(6, 1),
(6, 4),
(6, 1),
(6, 1),
(6, 6),
(20, 10),
(20, 5),
(20, 8),
(20, 3),
(20, 10),
(20, 6),
(20, 1),
(20, 12),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 2),
(2, 1),
(2, 2),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(4, 2),
(4, 2),
(4, 1),
(4, 0),
(4, 2),
(4, 1),
(4, 2),
(4, 3),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 0),
(2, 0),
(2, 2),
(6, 1),
(6, 2),
(6, 2),
(6, 4),
(6, 1),
(6, 2),
(6, 0),
(6, 3),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(4, 2),
(4, 3),
(4, 3),
(4, 2),
(4, 3),
(4, 2),
(4, 2),
(4, 1),
(4, 2),
(4, 1),
(4, 4),
(4, 1),
(4, 2),
(4, 1),
(4, 3),
(4, 2),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 1),
(2, 0),
(2, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 0),
(2, 0),
(2, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 2),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 1),
(2, 0),
(2, 0),
(2, 1),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0),
(0, 0)]

type3right = [i[1] for i in type3right_manual]

eval_table['Type 3 Right Manual'] = type3right

# add vision problems
years = ['2010v','2011v']

models = ["gpt4o", "claude3.5s", "claude3o", "claude3h", "gemini1.5p", "gpt4", "gpt4om"]

eval_table_v = pd.DataFrame(columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible","Type 2 Right", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])


for year in years:
    for model in models:
        if year == '2010':
            path = '../iol_2010/' + model + '/evaluation_3.txt'
        else:
            path = '../iol_2011/' + model + '/evaluation_5.txt'
        json_obj = text_to_json(path)
        if json_obj:
            total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right, type2_possible, type3_right, type3_possible = vision_prob_parser(json_obj)

            if accuracy > 0.75:
                print(f"Processing year {year} for model {model}")
                print(f"Total Score: {total_score}/{total_possible}")
                print(f"Accuracy: {accuracy}")
            
            
            eval_table_v = pd.concat([eval_table_v, pd.DataFrame([[year, model, total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right,  type2_possible, type3_right, type3_possible]], columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible", "Type 2 Right", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])], ignore_index=True)
        else:
            print(f"Error processing year {year} for model {model}")


In [7]:
# add vision problems
years = ['2010v','2011v']

models = ["gpt4o", "claude3.5s", "claude3o", "claude3h", "gemini1.5p", "gpt4", "gpt4om"]

eval_table_v = pd.DataFrame(columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible","Type 2 Right", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])


for year in years:
    for model in models:
        if year == '2010':
            path = '../iol_2010/' + model + '/evaluation_3.txt'
        else:
            path = '../iol_2011/' + model + '/evaluation_5.txt'
        json_obj = text_to_json(path)
        if json_obj:
            total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right, type2_possible, type3_right, type3_possible = vision_prob_parser(json_obj)

            if accuracy > 0.75:
                print(f"Processing year {year} for model {model}")
                print(f"Total Score: {total_score}/{total_possible}")
                print(f"Accuracy: {accuracy}")
            
            
            eval_table_v = pd.concat([eval_table_v, pd.DataFrame([[year, model, total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right,  type2_possible, type3_right, type3_possible]], columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible", "Type 2 Right", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])], ignore_index=True)
        else:
            print(f"Error processing year {year} for model {model}")


/var/folders/5t/bmhkrbcn4yzgbdrrgwk7ps_r0000gn/T/ipykernel_90267/3290123000.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eval_table_v = pd.concat([eval_table_v, pd.DataFrame([[year, model, total_score, total_possible, accuracy, total_questions, type1_right, type1_possible, type2_right,  type2_possible, type3_right, type3_possible]], columns=["Year", "Model", "Total Score", "Total Possible", "Accuracy", "Total Questions", "Type 1 Right", "Type 1 Possible", "Type 2 Right", "Type 2 Possible", "Type 3 Right", "Type 3 Possible"])], ignore_index=True)


In [9]:
eval_table.to_csv("eval_table.csv")

In [10]:
eval_table_v.to_csv("eval_table_v.csv")

In [9]:
eval_table

,Year,Model,Total Score,Total Possible,Accuracy,Total Questions,Type 1 Right,Type 1 Possible,Type 2 Right LLM,Type 2 Right Diff,Type 2 Right Bleu,Type 2 Possible,Type 3 Right,Type 3 Possible,Type 3 Right Manual
0,2004,gpt4o,19.981644,45,0.444037,5,3,13,7,15.981644,3.286918,24,1,8,1
1,2004,claude3.5s,22.132561,45,0.491835,5,5,13,11,16.132561,2.271757,24,1,8,1
2,2004,claude3o,26.834697,45,0.596327,5,5,13,15,18.834697,3.298447,24,3,8,3
3,2004,claude3h,22.951422,45,0.510032,5,2,13,12,16.951422,3.960966,24,4,8,4
4,2004,gemini1.5p,17.979579,45,0.399546,5,2,13,5,12.979579,0.822267,24,3,8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,samples,claude3h,116.445351,204,0.570811,12,43,94,49,73.445351,26.635306,110,0,0,0
156,samples,gemini1.5p,111.465203,190,0.586659,12,35,80,58,76.465203,17.512947,110,0,0,0
157,samples,gpt4,113.291018,212,0.534392,12,33,94,46,80.291018,3.352702,118,0,0,0
158,samples,gpt4om,68.376404,183,0.373642,12,17,91,28,51.376404,5.337481,92,0,0,0


In [8]:
eval_table_v

,Year,Model,Total Score,Total Possible,Accuracy,Total Questions,Type 1 Right,Type 1 Possible,Type 2 Right,Type 2 Possible,Type 3 Right,Type 3 Possible
0,2010v,gpt4o,4,16,0.2500,1,2,6,2,10,0,0
1,2010v,claude3.5s,4,16,0.2500,1,1,6,3,10,0,0
2,2010v,claude3o,2,16,0.1250,1,1,6,1,10,0,0
3,2010v,claude3h,0,16,0.0000,1,0,6,0,10,0,0
4,2010v,gemini1.5p,4,16,0.2500,1,2,6,2,10,0,0
5,2010v,gpt4,1,16,0.0625,1,1,6,0,10,0,0
6,2010v,gpt4om,2,16,0.1250,1,1,6,1,10,0,0
7,2011v,gpt4o,4,16,0.2500,1,2,6,2,10,0,0
8,2011v,claude3.5s,4,16,0.2500,1,1,6,3,10,0,0
9,2011v,claude3o,2,16,0.1250,1,1,6,1,10,0,0


In [9]:
# Calculate the percentage of Type 1 Right for each row
eval_table_v['Type 1 Percentage'] = eval_table_v['Type 1 Right'] / eval_table_v['Type 1 Possible']

# Group by 'Model' and calculate the mean percentage of Type 1 Right for each model
type1_percentage_by_model_v = eval_table_v.groupby('Model')['Type 1 Percentage'].mean()

# Display the result
print(type1_percentage_by_model_v)

Model
claude3.5s    0.166667
claude3h           0.0
claude3o      0.166667
gemini1.5p    0.333333
gpt4          0.166667
gpt4o         0.333333
gpt4om        0.166667
Name: Type 1 Percentage, dtype: object


In [10]:
eval_table_v['Type 2 Percentage'] = eval_table_v['Type 2 Right'] / eval_table_v['Type 2 Possible']

type2_percentage_by_model_v = eval_table_v.groupby('Model')['Type 2 Percentage'].mean()

print(type2_percentage_by_model_v)

Model
claude3.5s    0.3
claude3h      0.0
claude3o      0.1
gemini1.5p    0.2
gpt4          0.0
gpt4o         0.2
gpt4om        0.1
Name: Type 2 Percentage, dtype: object


In [12]:
eval_table_v['Type 3 Percentage'] = eval_table_v.apply(
    lambda row: (row['Type 3 Right'] / row['Type 3 Possible']) * 100 if row['Type 3 Possible'] > 0 else
    0, axis=1
)
type3_percentage_by_model_v = eval_table_v.groupby('Model')['Type 3 Percentage'].mean()

print(type3_percentage_by_model_v)

Model
claude3.5s    0.0
claude3h      0.0
claude3o      0.0
gemini1.5p    0.0
gpt4          0.0
gpt4o         0.0
gpt4om        0.0
Name: Type 3 Percentage, dtype: float64


In [29]:
# Calculate the percentage of Type 1 Right for each row
eval_table['Type 1 Percentage'] = (eval_table['Type 1 Right'] / eval_table['Type 1 Possible']) * 100

# Group by 'Model' and calculate the mean percentage of Type 1 Right for each model
type1_percentage_by_model = eval_table.groupby('Model')['Type 1 Percentage'].mean()

# Display the result
print(type1_percentage_by_model)

Model
claude3.5s      37.471159
claude3h         18.11579
claude3o        37.031134
gemini1.5p      18.896007
gpt4            21.384711
gpt4o           27.301799
gpt4om          21.553478
o1strawberry    28.459939
Name: Type 1 Percentage, dtype: object


In [30]:
# Calculate the percentage of Type 1 Right for each row

eval_table['Type 2 Percentage LLM Graded'] = eval_table.apply(
    lambda row: (row['Type 2 Right LLM'] / row['Type 2 Possible']) * 100 if row['Type 2 Right LLM'] > 0 else 0,
    axis=1
)

eval_table['Type 2 Right Bleu Score'] = eval_table.apply(
    lambda row: (row['Type 2 Right Bleu'] / row['Type 2 Possible']) * 100 if row['Type 2 Right Bleu'] > 0 else 0,
    axis=1
)

eval_table['Type 2 Right Diff Score'] = eval_table.apply(
    lambda row: (row['Type 2 Right Diff'] / row['Type 2 Possible']) * 100 if row['Type 2 Right Diff'] > 0 else 0,
    axis=1
)

# Group by 'Model' and calculate the mean percentage of Type 1 Right for each model
type2_llm_percentage_by_model = eval_table.groupby('Model')['Type 2 Percentage LLM Graded'].mean()

type2_bleu_percentage_by_model = eval_table.groupby('Model')['Type 2 Right Bleu Score'].mean()

type2_diff_percentage_by_model = eval_table.groupby('Model')['Type 2 Right Diff Score'].mean()

# Display the result
print(type2_llm_percentage_by_model)

print(type2_bleu_percentage_by_model)

print(type2_diff_percentage_by_model)

Model
claude3.5s      59.914754
claude3h        43.009400
claude3o        48.733995
gemini1.5p      40.471049
gpt4            37.701040
gpt4o           39.252441
gpt4om          31.172195
o1strawberry    46.879788
Name: Type 2 Percentage LLM Graded, dtype: float64
Model
claude3.5s      22.861759
claude3h        17.180799
claude3o        10.909842
gemini1.5p       7.750653
gpt4            10.905816
gpt4o            7.300873
gpt4om           6.200295
o1strawberry    10.414871
Name: Type 2 Right Bleu Score, dtype: float64
Model
claude3.5s      76.984018
claude3h        63.100367
claude3o        62.365016
gemini1.5p      57.564699
gpt4            63.887071
gpt4o           60.831881
gpt4om          54.183915
o1strawberry    65.893937
Name: Type 2 Right Diff Score, dtype: float64


In [ ]:
# remove rows with 0 possible questions

eval_table_c = eval_table[eval_table['Type 3 Possible'] > 0].copy()

eval_table_c['Type 3 Percentage LLM Graded'] = eval_table_c.apply(
    lambda row: (row['Type 3 Right'] / row['Type 3 Possible']) * 100 if row['Type 3 Possible'] > 0 else
    0, axis=1
)

eval_table_c['Type 3 Percentage Manual Graded'] = eval_table_c.apply(
    lambda row: (row['Type 3 Right Manual'] / row['Type 3 Possible']) * 100 if row['Type 3 Possible'] > 0 else 0,
    axis=1
)


# Group by 'Model' and calculate the mean percentage of Type 1 Right for each model
type3_llm_percentage_by_model = eval_table_c.groupby('Model')['Type 3 Percentage LLM Graded'].mean()

type3_man_percentage_by_model = eval_table_c.groupby('Model')['Type 3 Percentage Manual Graded'].mean()


# Display the result
print(type3_llm_percentage_by_model)
print(type3_man_percentage_by_model)


Model
claude3.5s      38.782051
claude3h        40.256410
claude3o        44.423077
gemini1.5p      45.833333
gpt4            31.153846
gpt4o           41.987179
gpt4om          30.512821
o1strawberry    33.974359
Name: Type 3 Percentage LLM Graded, dtype: float64
Model
claude3.5s      38.782051
claude3h        40.256410
claude3o        44.423077
gemini1.5p      45.833333
gpt4            31.153846
gpt4o           41.987179
gpt4om          26.666667
o1strawberry    58.461538
Name: Type 3 Percentage Manual Graded, dtype: float64
